In [1]:
import datetime
import hashlib
import json

In [2]:
class Blockchain:
    def __init__(self):
        self.chain = []
        self.transactions = [] # Initialize transactions list
        self.create_block(proof = 1, previous_hash = '0', merkle_root = '0', transactions=[])

    def create_block(self, proof, previous_hash, merkle_root, transactions):
        block = {'index': len(self.chain) + 1,
                 'timestamp': str(datetime.datetime.now()),
                 'proof': proof,
                 'previous_hash': previous_hash,
                 'merkle_root': merkle_root,
                 'transactions': transactions # Include transactions in the block
                }
        self.chain.append(block)
        return block

    def get_merkle_root(self, transactions):
        if not transactions:
            return '0'
        
        # Extract data to hash (using full transaction dict strings)
        temp_transactions = [json.dumps(tx, sort_keys=True) for tx in transactions]
        
        while len(temp_transactions) > 1:
            if len(temp_transactions) % 2 != 0:
                temp_transactions.append(temp_transactions[-1])
            
            new_level = []
            for i in range(0, len(temp_transactions), 2):
                hash_val = hashlib.sha256((temp_transactions[i] + temp_transactions[i+1]).encode()).hexdigest()
                new_level.append(hash_val)
            temp_transactions = new_level
            
        return temp_transactions[0]

    def get_previous_block(self):
        return self.chain[-1]

    def proof_of_work(self, previous_hash, merkle_root):
        new_proof = 0
        check_proof = False
        while check_proof is False:
            # Hash operation on relevant block data
            # For simplicity in this demo, we combine previous_hash, merkle_root and proof/nonce
            hash_operation = hashlib.sha256(str(str(new_proof) + previous_hash + merkle_root).encode()).hexdigest()
            if hash_operation[:3] == '000': 
                check_proof = True
            else:
                new_proof += 1
        return new_proof

    def hash(self, block):
        encoded_block = json.dumps(block, sort_keys = True).encode()
        return hashlib.sha256(encoded_block).hexdigest()

    def is_chain_valid(self, chain):
        previous_block = chain[0]
        block_index = 1
        while block_index < len(chain):
            block = chain[block_index]
            if block['previous_hash'] != self.hash(previous_block):
                return False
            
            # Verify Merkle Root
            transactions = block['transactions']
            if self.get_merkle_root(transactions) != block['merkle_root']:
                print(f"Merkle Root Mismatch at Block {block_index}")
                return False

            # Verify Proof of Work (Golden Nonce)
            previous_hash = block['previous_hash']
            merkle_root = block['merkle_root']
            proof = block['proof']
            
            hash_operation = hashlib.sha256(str(str(proof) + previous_hash + merkle_root).encode()).hexdigest()
            
            if hash_operation[:3] != '000':
                print(f"Invalid Proof at Block {block_index}")
                return False
                
            previous_block = block
            block_index += 1
        return True

    def create_transaction(self, sender, receiver, amount):
        self.transactions.append({
            'sender': sender,
            'receiver': receiver,
            'amount': amount
        })
        # Returns index of the block it *will* be added to (next one)
        return self.get_previous_block()['index'] + 1

In [3]:
# Creating a Blockchain
blockchain = Blockchain()

In [4]:
# Mining a new block
def mine_block():
    if len(blockchain.transactions) < 5:
        return {'message': 'Not enough transactions to mine (min 5). Current: ' + str(len(blockchain.transactions))}

    # Select first 5 transactions
    transactions_to_mine = blockchain.transactions[:5]
    # Remove them from mempool
    blockchain.transactions = blockchain.transactions[5:]

    previous_block = blockchain.get_previous_block()
    previous_hash = blockchain.hash(previous_block)
    merkle_root = blockchain.get_merkle_root(transactions_to_mine)

    proof = blockchain.proof_of_work(previous_hash, merkle_root)
   
    block = blockchain.create_block(proof, previous_hash, merkle_root, transactions_to_mine)
    
    response = {'message': 'Congratulations, you just mined a block!',
                'index': block['index'],
                'timestamp': block['timestamp'],
                'proof': block['proof'],
                'previous_hash': block['previous_hash'],
                'merkle_root': block['merkle_root'],
                'transactions': block['transactions'] # Include mined transactions in response
               }
    return response

In [5]:
# Getting the full Blockchain
def get_chain():
    response = {'chain': blockchain.chain,
                'length': len(blockchain.chain)}
    return response

In [6]:
# Checking if the Blockchain is valid
def is_valid():
    is_valid = blockchain.is_chain_valid(blockchain.chain)
    if is_valid:
        response = {'message': 'All good. The Blockchain is valid.'}
    else:
        response = {'message': 'Houston, we have a problem. The Blockchain is not valid.'}
    return response

In [ ]:
import re

def add_transaction():
    sender = input("Enter sender: ")
    receiver = input("Enter receiver: ")
    amount_str = input("Enter amount: ")
    # Extract numerical part from the string using regex
    numerical_amount = re.findall(r'\d+\.?\d*', amount_str)
    if numerical_amount:
        amount = float(numerical_amount[0])
    else:
        print("Invalid amount format. Please enter a valid number.")
        return

    index = blockchain.create_transaction(sender=sender, receiver=receiver, amount=amount)
    response = {'message': f'Transaction will be added to Block {index}'}
    return response

while True:
    print("\nFunctions Menu:")
    print("===============")
    print("1. Mine a block")
    print("2. Display the chain")
    print("3. Check the validity of the chain")
    print("4. Add a transaction")
    print("5. Exit")
    try:
        choice = int(input("Enter your choice :"))
    except ValueError:
        print("Invalid choice")
        continue

    if choice == 1:
        print(mine_block())
    elif choice == 2:
        print(get_chain())
    elif choice == 3:
        print(is_valid())
    elif choice == 4:
        print(add_transaction())
    elif choice == 5:
        print("Exiting...")
        break
    else :
        print("Invalid choice")


Functions Menu:
1. Mine a block
2. Display the chain
3. Check the validity of the chain
4. Add a transaction
5. Exit
Invalid choice

Functions Menu:
1. Mine a block
2. Display the chain
3. Check the validity of the chain
4. Add a transaction
5. Exit
Invalid choice

Functions Menu:
1. Mine a block
2. Display the chain
3. Check the validity of the chain
4. Add a transaction
5. Exit
{'message': 'Not enough transactions to mine (min 5). Current: 0'}

Functions Menu:
1. Mine a block
2. Display the chain
3. Check the validity of the chain
4. Add a transaction
5. Exit
{'message': 'Transaction will be added to Block 2'}

Functions Menu:
1. Mine a block
2. Display the chain
3. Check the validity of the chain
4. Add a transaction
5. Exit
{'chain': [{'index': 1, 'timestamp': '2026-02-09 09:45:49.895019', 'proof': 1, 'previous_hash': '0', 'merkle_root': '0', 'transactions': []}], 'length': 1}

Functions Menu:
1. Mine a block
2. Display the chain
3. Check the validity of the chain
4. Add a transac